<a href="https://colab.research.google.com/github/alenelias123/Compliment-Generator/blob/main/Compliment_Generator_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio transformers torch opencv-python scikit-learn

import gradio as gr
from transformers import pipeline
import cv2
import numpy as np
from sklearn.cluster import KMeans
from PIL import Image

generator = pipeline("text-generation", model="gpt2")

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
smile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')

def generate_compliment(image):
    img = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.2, 5)
    is_smiling = False

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        smiles = smile_cascade.detectMultiScale(roi_gray, scaleFactor=1.9, minNeighbors=20)
        if len(smiles) > 0:
            is_smiling = True
            break

    h, w, _ = img.shape
    center_y, center_x = h // 2, w // 2
    clothing_region = img[center_y-50:center_y+50, center_x-50:center_x+50]
    pixels = clothing_region.reshape(-1, 3)
    kmeans = KMeans(n_clusters=1, random_state=0).fit(pixels)
    dominant_color = kmeans.cluster_centers_[0].astype(int)

    prompt = "You have a wonderful "
    if is_smiling:
        prompt += "smile that could brighten anyone's day. "
    if is_smiling == False:
        prompt+="smile but you dont intend to show it to anyone. "
    color_name = "stylish outfit"
    if np.allclose(dominant_color, [50, 50, 255], atol=100):
        color_name = "stunning red outfit"
    elif np.allclose(dominant_color, [50, 255, 50], atol=100):
        color_name = "gorgeous green attire"
    elif np.allclose(dominant_color, [255, 50, 50], atol=100):
        color_name = "beautiful blue ensemble"
    prompt += f"Your {color_name} really suits you!"
    response = generator(prompt, max_length=50, temperature=0.9, num_return_sequences=2)

    return response[0]["generated_text"]

iface = gr.Interface(
    fn=generate_compliment,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="Over-the-Top Compliment Generator",
    description="Upload an image to receive a customized compliment based on your smile and outfit color."
)


iface.launch()
